In [1]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.common.by import By
import mysql.connector

In [2]:
#Conexão incial com o banco de dados
conexao =  mysql.connector.connect(host='localhost',user='root',password='root')
print('Conexão com o banco de dados realizada com sucesso!')

#Criação do Schema e das tabelas do banco de dados
cursor = conexao.cursor()
schema = """CREATE SCHEMA IF NOT EXISTS `Raspagem` DEFAULT CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;"""
tables = """use raspagem;
CREATE TABLE IF NOT EXISTS `Raspagem`.`Tipo_Vaga` (
            ID Int not null Auto_increment,
            Tipo Varchar(50),
            Primary Key (ID)
             ) Engine = InnoDB;

CREATE TABLE IF NOT EXISTS `Raspagem`.`Vagas` (
            ID INT NOT NULL AUTO_INCREMENT,
            Vaga VARCHAR(200) NOT NULL,
            Empresa VARCHAR(200) NOT NULL,
            Local VARCHAR(200) NOT NULL,
            Data DATE NOT NULL,
            Descricao MEDIUMTEXT NOT NULL,
            URL VARCHAR(500) NOT NULL,
            UltimoAcesso DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP,
            TipoVaga INT NOT NULL, 
            Primary Key (ID),
            FOREIGN KEY (TipoVaga) REFERENCES Tipo_Vaga(ID)
            ) ENGINE = InnoDB;
        """
cursor.execute(schema)
cursor.execute('use Raspagem')
print('Schema criado com sucesso!')
cursor.execute(tables)
print('Tabelas criadas com sucesso!')
conexao.close()

conexao = mysql.connector.connect(host='localhost', database= 'Raspagem',user='root', password='root')
cursor = conexao.cursor()

Conexão com o banco de dados realizada com sucesso!
Schema criado com sucesso!
Tabelas criadas com sucesso!


In [3]:
#Reconexão com o banco de dados
conexao = mysql.connector.connect(host='localhost', database= 'Raspagem',user='root', password='root')
cursor = conexao.cursor()

In [4]:
#Inserção dos tipos de vagas
cursor.execute("insert into tipo_vaga (Tipo) values ('Estágio');")
cursor.execute("insert into tipo_vaga (Tipo) values ('Assistente');")
cursor.execute("insert into tipo_vaga (Tipo) values ('Júnior');")
cursor.execute("insert into tipo_vaga (Tipo) values ('Pleno-Sênior');")
cursor.execute("insert into tipo_vaga (Tipo) values ('Diretor');")

In [8]:
#Raspagem Oficial
#Setup para a raspagem
#A URL deve ser mudada de acordo com os filtros feitos no linkedin
url = 'https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=&location=Brasil&locationId=&geoId=106057199&f_TPR=&f_E=2&start=0'
binary = FirefoxBinary("C:\\Program Files\\Mozilla Firefox\\firefox.exe")
driver = webdriver.Firefox(firefox_binary=binary, executable_path="C:\\geckodriver.exe",)

#Raspagem das vagas
#A plataforma permite a visualização de 1000 vagas, depois da 1000 ele apresenta erro 404
for page in range (0, 100, 25):
    url = 'https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=&location=Brasil&locationId=&geoId=106057199&f_TPR=&f_E=2&start='+ str(page)
    driver.get(url)
    time.sleep(4)
    
    #Cada Página apresenta 25 vagas
    for i in range(1, 26):
        try:
            #Coleta dos dados da vaga
            time.sleep(2)
            NomeVaga = driver.find_element(By.XPATH, '/html/body/li[' + str(i) + ']/div/div[2]/h3').text
            Empresa = driver.find_element(By.XPATH, '/html/body/li[' + str(i) + ']/div/div[2]/h4/a').text
            Local = driver.find_element(By.XPATH, '/html/body/li['  + str(i) + ']/div/div[2]/div/span').text 
            Data = driver.find_element(By.XPATH, '/html/body/li['   + str(i) + ']/div/div[2]/div/time').get_attribute('datetime')
            URL = driver.find_element(By.XPATH, '/html/body/li[' + str(i) + ']/div/a').get_attribute('href')

            #Acessa a página da vaga e coleta a descrição
            driver.find_element(By.XPATH, '/html/body/li['+ str(i) + ']/div/a/span').click() 
            time.sleep(2)
            driver.find_element(By.XPATH, '//*[@id="main-content"]/section[1]/div/div/section[1]/div/div/section/button[1]').click()
            Desc = driver.find_element(By.XPATH, '//*[@id="main-content"]/section[1]/div/div/section[1]/div/div/section/div').text
            driver.back()
            time.sleep(2)
            
            #Verifica se a vaga já está cadastrada
            testador = False
            cursor.execute('SELECT * FROM Vagas WHERE Vaga = %s AND Empresa = %s AND Local = %s AND Data = %s AND descricao = %s', (NomeVaga, Empresa, Local, Data, Desc))
            result = cursor.fetchone()
            if result:
                print(f"A vaga {NomeVaga} já está cadastrada.")
                testador = True

            #Valida e caso não esteja, registra a vaga no banco
            if testador == False:   
                cursor.execute('INSERT INTO Vagas (Vaga, Empresa, Local, Data, descricao, URL,TipoVaga) VALUES (%s, %s, %s, %s, %s , %s, 2)', (NomeVaga, Empresa, Local, Data, Desc, URL))  
                conexao.commit()
            
        #Tratamento de erros
        #Caso ocorra algum erro de carregamento ou redirecionamento da página, ele recomeça a raspagem da vaga atual
        except Exception as e:
            print(f"Erro ao obter dados da vaga {i}. Tentando novamente...")
            print('erro: ', e)
            i -= 1
            driver.get(url)
            time.sleep(2)
            continue
    
    driver.close()

C:\Users\gabriel.ponzio\AppData\Local\Temp\ipykernel_18824\1934112101.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(firefox_binary=binary, executable_path="C:\\geckodriver.exe",)
C:\Users\gabriel.ponzio\AppData\Local\Temp\ipykernel_18824\1934112101.py:6: DeprecationWarning: firefox_binary has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(firefox_binary=binary, executable_path="C:\\geckodriver.exe",)


A vaga assistente administrativo já está cadastrada.
A vaga Balconista de Farmácia - João XXIII já está cadastrada.
A vaga Balconista de Farmácia - João XXIII já está cadastrada.
A vaga Recepcionista já está cadastrada.
A vaga assistente administrativo já está cadastrada.
A vaga balconista já está cadastrada.
A vaga Acompanhante de Idoso já está cadastrada.
A vaga Acompanhante de Idoso já está cadastrada.
A vaga Atendente de Adega já está cadastrada.
A vaga auxiliar administrativo já está cadastrada.
A vaga operador de caixa já está cadastrada.
A vaga auxiliar de loja já está cadastrada.
A vaga Auxiliar de Serviços Gerais - Limpeza já está cadastrada.
A vaga vendedor já está cadastrada.


InvalidSessionIdException: Message: Tried to run command without establishing a connection


# Codígo banco de dados + link das vagas com os filtros

1 - Link dos Estágios ->'https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=&location=Brasil&locationId=&geoId=106057199&f_TPR=&f_E=1&start=0'

2 - Link dos Assistentes -> https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=&location=Brasil&locationId=&geoId=106057199&f_TPR=&f_E=2&start=0

3 - Link dos Júniors -> https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=&location=Brasil&locationId=&geoId=106057199&f_TPR=&f_E=3&start=0

4 - Link dos Plenos-Sêniors -> https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=&location=Brasil&locationId=&geoId=106057199&f_TPR=&f_E=4&start=0

5 - Link dos Diretores -> https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=&location=Brasil&locationId=&geoId=106057199&f_TPR=&f_E=5&start=25
